In [1]:
import os

In [2]:
%pwd

'd:\\aiadventures\\Projects\\ml_complete_proj\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\aiadventures\\Projects\\ml_complete_proj'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file:Path
    unzip_dir: Path


In [15]:
from ml_complete_proj.constants import *
from  ml_complete_proj.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath  = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [17]:
import os
import urllib.request as request
import zipfile
from ml_complete_proj import logger
from ml_complete_proj.utils.common import get_size

In [18]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config  = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2024-12-25 13:50:34,730: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-25 13:50:34,745: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-25 13:50:34,750: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-25 13:50:34,753: INFO: common: created directory at: artifacts]
[2024-12-25 13:50:34,755: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-25 13:50:37,615: INFO: 2968097854: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 11939
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "1a94a2e500c6a3d4655ed5e5b3bc24bf8cbf13df88bd469e1bbe4c8e53cbd5f2"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C93C:25E8A1:4B3E4:776F4:676BC051
Accept-Ranges: bytes
Date: Wed, 